# k-Nearest Neighbor (kNN) mini-Project

#### This jupyter notebook mini-Project exercise is a simplified version modified from Assignment 1 of *Stanford CS class CS231n: Convolutional Neural Networks for Visual Recognition* by changing the input data from the CIFAR10 dataset to the MNIST dataset.. For more details see the [course assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.

The kNN classifier consists of two stages:

- During training, the classifier takes the training data and simply remembers it
- During testing, kNN classifies every test image by comparing to all training images and transfering the labels of the k most similar training examples
- The value of k is cross-validated

In this simplified exercise you will learn how to write efficient, vectorized code. 

In [ ]:
# Run some setup code for this notebook.

import random
import numpy as np
import matplotlib.pyplot as plt

from __future__ import print_function

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Load the dataset from http://deeplearning.net/data/mnist/mnist.pkl.gz
import urllib, os
str_dataset = 'mnist.pkl.gz'
if not os.path.isfile(str_dataset):
    print('Downloading mnist dataset...')
    url = 'http://deeplearning.net/data/mnist/mnist.pkl.gz'
    urllib.urlretrieve(url, str_dataset)
    print('Download complete')  

In [ ]:
# Load the dataset to traing and test sets
import sys, gzip
f = gzip.open(str_dataset, 'rb')
if sys.version_info[0] < 3:
    print("using python 2")
    import cPickle
    train_set, valid_set, test_set = cPickle.load(f)
else:
    print("using python 3")
    import _pickle as cPickle
    train_set, valid_set, test_set = cPickle.load(f,encoding='latin1')
f.close()

In [ ]:
print(train_set[0].shape,valid_set[0].shape, test_set[0].shape)
print(train_set[1].shape,valid_set[1].shape, test_set[1].shape)
X_train, y_train, X_test, y_test = train_set[0], train_set[1], test_set[0], test_set[1]

# f = np.load('cs231n/datasets/mnist.npz')
# X_train, y_train = f['x_train'], f['y_train']
# X_test, y_test = f['x_test'], f['y_test']

print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
#plt.imshow(train_set[0][i].reshape((28, 28))) #, cmap=cm.gray, interpolation='none')
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
num_classes = len(classes)
samples_per_class = 7
plt.figure(figsize=(10,10))
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].reshape((28, 28)))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Subsample the data for more efficient code execution in this exercise
num_training = 5000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 500
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

In [ ]:
# Reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

## FIXME: <font color='blue'>Complete the KNearestNeighbor class below. Concretely, you just need to complete the sections in the TODO parts</font>

We would now like to classify the test data with the kNN classifier. Recall that we can break down this process into two steps: 

1. First we must compute the distances between all test examples and all train examples. 
2. Given these distances, for each test example we find the k nearest examples and have them vote for the label

Lets begin with computing the distance matrix between all training and test examples. For example, if there are **Ntr** training examples and **Nte** test examples, this stage should result in a **Nte x Ntr** matrix where each element (i,j) is the distance between the i-th test and j-th train example.

In the KNearestNeighbor class:

1) implement the function `compute_distances_two_loops` that uses a (very inefficient) double loop over all pairs of (test, train) examples and computes the distance matrix one element at a time.

2) implement the function `compute_distances_one_loop` that uses a single loop over all pairs of (test, train) examples and computes the distance matrix one element at a time.

3) implement the function `compute_distances_no_loops` that uses no loop to compute the distance matrix one element at a time.


In [ ]:
class KNearestNeighbor(object):
  """ a kNN classifier with L2 distance """

  def __init__(self):
    pass

  def train(self, X, y):
    """
    Train the classifier. For k-nearest neighbors this is just 
    memorizing the training data.

    Inputs:
    - X: A numpy array of shape (num_train, D) containing the training data
      consisting of num_train samples each of dimension D.
    - y: A numpy array of shape (N,) containing the training labels, where
         y[i] is the label for X[i].
    """
    self.X_train = X
    self.y_train = y
    self.close_k = np.empty
    
  def predict(self, X, k=1, num_loops=0):
    """
    Predict labels for test data using this classifier.

    Inputs:
    - X: A numpy array of shape (num_test, D) containing test data consisting
         of num_test samples each of dimension D.
    - k: The number of nearest neighbors that vote for the predicted labels.
    - num_loops: Determines which implementation to use to compute distances
      between training points and testing points.

    Returns:
    - y: A numpy array of shape (num_test,) containing predicted labels for the
      test data, where y[i] is the predicted label for the test point X[i].  
    """
    if num_loops == 0:
      dists = self.compute_distances_no_loops(X)
    elif num_loops == 1:
      dists = self.compute_distances_one_loop(X)
    elif num_loops == 2:
      dists = self.compute_distances_two_loops(X)
    else:
      raise ValueError('Invalid value %d for num_loops' % num_loops)

    return self.predict_labels(dists, k=k)

  def compute_distances_two_loops(self, X):
    """
    Compute the distance between each test point in X and each training point
    in self.X_train using a nested loop over both the training data and the 
    test data.

    Inputs:
    - X: A numpy array of shape (num_test, D) containing test data.

    Returns:
    - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
      is the Euclidean distance between the ith test point and the jth training
      point.
    """
    num_test = X.shape[0]
    num_train = self.X_train.shape[0]
    dists = np.zeros((num_test, num_train))
    for i in xrange(num_test):
      for j in xrange(num_train):
        #####################################################################
        # TODO:                                                             #
        # Compute the l2 distance between the ith test point and the jth    #
        # training point, and store the result in dists[i, j]. You should   #
        # not use a loop over dimension.                                    #
        #####################################################################
        dists[i,j]=np.sqrt(np.sum(np.square(X[i,:] - self.X_train[j,:])))
        #pass
        #####################################################################
        #                       END OF YOUR CODE                            #
        #####################################################################
    return dists

  def compute_distances_one_loop(self, X):
    """
    Compute the distance between each test point in X and each training point
    in self.X_train using a single loop over the test data.

    Input / Output: Same as compute_distances_two_loops
    """
    num_test = X.shape[0]
    num_train = self.X_train.shape[0]
    dists = np.zeros((num_test, num_train))
    for i in xrange(num_test):
      #######################################################################
      # TODO:                                                               #
      # Compute the l2 distance between the ith test point and all training #
      # points, and store the result in dists[i, :].                        #
      #######################################################################
      dists[i]=np.sqrt(np.sum(np.square(X[i,:]-self.X_train),axis=1))
      # pass
      #######################################################################
      #                         END OF YOUR CODE                            #
      #######################################################################
    return dists

  def compute_distances_no_loops(self, X):
    """
    Compute the distance between each test point in X and each training point
    in self.X_train using no explicit loops.

    Input / Output: Same as compute_distances_two_loops
    """
    num_test = X.shape[0]
    num_train = self.X_train.shape[0]
    dists = np.zeros((num_test, num_train)) 
    #########################################################################
    # TODO:                                                                 #
    # Compute the l2 distance between all test points and all training      #
    # points without using any explicit loops, and store the result in      #
    # dists.                                                                #
    #                                                                       #
    # You should implement this function using only basic array operations; #
    # in particular you should not use functions from scipy.                #
    #                                                                       #
    # HINT: Try to formulate the l2 distance using matrix multiplication    #
    #       and two broadcast sums.                                         #
    #########################################################################
    teX2=np.sum(X**2,axis=1).reshape(num_test,-1)
    trX2=np.sum(self.X_train**2,axis=1)
    trte2=np.dot(X,self.X_train.T)
    dists=np.sqrt(teX2+trX2-2.0*trte2)
    #pass
    #########################################################################
    #                         END OF YOUR CODE                              #
    #########################################################################
    return dists

  def predict_labels(self, dists, k=1):
    """
    Given a matrix of distances between test points and training points,
    predict a label for each test point.

    Inputs:
    - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
      gives the distance betwen the ith test point and the jth training point.

    Returns:
    - y_pred: A numpy array of shape (num_test,) containing predicted labels for the
      test data, where y[i] is the predicted label for the test point X[i].  
    - close_k: A list of numpy array containing the k nearest neighbor of the test data
    """
    num_test = dists.shape[0]
    y_pred = np.zeros(num_test,dtype=int)
    close_k = np.zeros((num_test,k),dtype=int)
    for i in xrange(num_test):
      # A list of length k storing the labels of the k nearest neighbors to
      # the ith test point.
      closest_y = []
      closest_y = np.argsort(dists[i,:])[:k]
      close_k[i] = closest_y
      y_pred[i] = np.bincount(self.y_train[closest_y]).argmax()
      #pass
      #########################################################################
      #                           END OF YOUR CODE                            # 
      #########################################################################

    return y_pred, close_k

In [ ]:
# Create a kNN classifier instance. 
# Remember that training a kNN classifier is a noop: 
# the Classifier simply remembers the data and does no further processing 
classifier = KNearestNeighbor()
classifier.train(X_train, y_train)

In [ ]:
# Open cs231n/classifiers/k_nearest_neighbor.py and implement
# compute_distances_two_loops.

# Test your implementation:
dists = classifier.compute_distances_two_loops(X_test)
print(dists.shape)

In [ ]:
# We can visualize the distance matrix: each row is a single test example and
# its distances to training examples
plt.imshow(dists, interpolation='none')
plt.show()

In [ ]:
# Now implement the function predict_labels and run the code below:
# We use k = 1 (which is Nearest Neighbor).
y_test_pred,close_k = classifier.predict_labels(dists, k=1)

# Compute and print the fraction of correctly predicted examples
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

You should expect to see approximately `90%` accuracy. Now lets try out a larger `k`, say `k = 5`:

In [ ]:
y_test_pred,close_k = classifier.predict_labels(dists, k=5)
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

You should expect to see a slightly better performance than with `k = 1`.

In [ ]:
# Now lets speed up distance matrix computation by using partial vectorization
# with one loop. Implement the function compute_distances_one_loop and run the
# code below:
dists_one = classifier.compute_distances_one_loop(X_test)

# To ensure that our vectorized implementation is correct, we make sure that it
# agrees with the naive implementation. There are many ways to decide whether
# two matrices are similar; one of the simplest is the Frobenius norm. In case
# you haven't seen it before, the Frobenius norm of two matrices is the square
# root of the squared sum of differences of all elements; in other words, reshape
# the matrices into vectors and compute the Euclidean distance between them.
difference = np.linalg.norm(dists - dists_one, ord='fro')
print('Difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [ ]:
# Now implement the fully vectorized version inside compute_distances_no_loops
# and run the code
dists_two = classifier.compute_distances_no_loops(X_test)

# check that the distance matrix agrees with the one we computed before:
difference = np.linalg.norm(dists - dists_two, ord='fro')
print('Difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [ ]:
# Let's compare how fast the implementations are
def time_function(f, *args):
    """
    Call a function f with args and return the time (in seconds) that it took to execute.
    """
    import time
    tic = time.time()
    f(*args)
    toc = time.time()
    return toc - tic

two_loop_time = time_function(classifier.compute_distances_two_loops, X_test)
print('Two loop version took %f seconds' % two_loop_time)

one_loop_time = time_function(classifier.compute_distances_one_loop, X_test)
print('One loop version took %f seconds' % one_loop_time)

no_loop_time = time_function(classifier.compute_distances_no_loops, X_test)
print('No loop version took %f seconds' % no_loop_time)

# you should see significantly faster performance with the fully vectorized implementation

### Cross-validation

We have implemented the k-Nearest Neighbor classifier but we set the value k = 5 arbitrarily. We will now determine the best value of this hyperparameter with cross-validation.

In [ ]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 20]

X_train_folds = []
y_train_folds = []
# Split up the training data into folds. After splitting, X_train_folds and   
# y_train_folds are lists of length num_folds, where               
# y_train_folds[i] is the label vector for the points in X_train_folds[i].    
X_train_folds = np.array_split(X_train, num_folds)
y_train_folds = np.array_split(y_train, num_folds)

# A dictionary holding the accuracies for different values of k that we find
# when running cross-validation. After running cross-validation,
# k_to_accuracies[k] should be a list of length num_folds giving the different
# accuracy values that we found when using that value of k.
k_to_accuracies = {}

################################################################################
# Perform k-fold cross validation to find the best value of k. For each        #
# possible value of k, run the k-nearest-neighbor algorithm num_folds times,   #
# where in each case you use all but one of the folds as training data and the #
# last fold as a validation set. Store the accuracies for all fold and all     #
# values of k in the k_to_accuracies dictionary.                               #
################################################################################
for k in k_choices:
    k_to_accuracies[k]=[]
    for f in range(num_folds):
        x_trn = []
        y_trn = []
        trn_idc = []
        for i in range(num_folds):
            if (i!=f):
                x_trn.append(X_train_folds[i])
                y_trn.append(y_train_folds[i])
        x_trn = np.stack(x_trn).reshape(-1,X_train.shape[1])
        y_trn = np.stack(y_trn).reshape(x_trn.shape[0])

        x_val = X_train_folds[f]
        y_val = y_train_folds[f]

        classifier.train(x_trn, y_trn)
        dists = classifier.compute_distances_no_loops(x_val)
        y_val_pred,close_k = classifier.predict_labels(dists,k)
        num_val = y_val.shape[0]
        num_corr = np.sum(y_val_pred == y_val)
        acc = float(num_corr) / num_val
        k_to_accuracies[k].append(acc)

# Print out the computed accuracies
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %2d, accuracy = %f' % (k, accuracy))

In [ ]:
# plot the raw observations
for k in k_choices:
    accuracies = k_to_accuracies[k]
    plt.scatter([k] * len(accuracies), accuracies)

# plot the trend line with error bars that correspond to standard deviation
accuracies_mean = np.array([np.mean(v) for k,v in sorted(k_to_accuracies.items())])
accuracies_std = np.array([np.std(v) for k,v in sorted(k_to_accuracies.items())])
plt.errorbar(k_choices, accuracies_mean, yerr=accuracies_std)
plt.title('Cross-validation on k')
plt.xlabel('k')
plt.ylabel('Cross-validation accuracy')
plt.show()

In [ ]:
# Based on the cross-validation results above, choose the best value for k,   
# retrain the classifier using all the training data, and test it on the test
# data. You should be able to get above 28% accuracy on the test data.
best_k = 7

classifier = KNearestNeighbor()
classifier.train(X_train, y_train)
y_test_pred,close_k = classifier.predict(X_test, k=best_k)

# Compute and display the accuracy
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

### Plot some samples of correctly labled test images

In [ ]:
nsample = 5
correct_indices = np.nonzero(y_test_pred == y_test)[0][:nsample]
ii =1
plt.figure(figsize=(12,12))
for inc_idx in correct_indices:
    # plot the test image
    plt.subplot(nsample, best_k+1, ii)
    plt.imshow(X_test[inc_idx].reshape((28, 28)))
    plt.title("Test:{},Pred:{}".format(y_test[inc_idx],y_test_pred[inc_idx]),color="red")
    plt.axis('off')
    ii+=1
    # plot the k neighbors
    for idx_close in close_k[inc_idx]:
        plt.subplot(nsample, best_k+1, ii)
        ii+=1
        plt.imshow(X_train[idx_close].reshape((28, 28)))
        plt.title("Train: {}".format(y_train[idx_close]),color="blue")
        plt.axis('off')
plt.tight_layout()
plt.show()

### Plot some samples of incorrectly labled test images

In [ ]:
nsample = 5
incorrect_indices = np.nonzero(y_test_pred != y_test)[0][:nsample]
ii =1
plt.figure(figsize=(12,12))
for inc_idx in incorrect_indices:
    # plot the test image
    plt.subplot(nsample, best_k+1, ii)
    plt.imshow(X_test[inc_idx].reshape((28, 28)))
    plt.title("Test:{}, Pred:{}".format(y_test[inc_idx],y_test_pred[inc_idx]),color="red") # Pred {:d},fontsize=10
    plt.axis('off')
    ii+=1
    # plot the k neighbors
    for idx_close in close_k[inc_idx]:
        plt.subplot(nsample, best_k+1, ii)
        ii+=1
        plt.imshow(X_train[idx_close].reshape((28, 28)))
        plt.title("Train: {}".format(y_train[idx_close]),color="blue")
        plt.axis('off')
plt.tight_layout()

plt.show()